# Rodar Teste 1: influência da SNR

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from typing import Optional, Any
from pathlib import Path
from time import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from joblib import Parallel, delayed

## Funções

In [2]:
def abrirDataframe(
    dirDatasets:        Path, 
    qtdUsuarios:        int, 
    qtdAntenas:         int,
    datasetTreinamento: bool
) -> pd.DataFrame:
    
    prefixo    = "train_1_" if datasetTreinamento else "test_1_"        
    caminhoCSV = sorted(dirDatasets.glob(prefixo+str(qtdUsuarios)+"_users_"+str(qtdAntenas)+"_antenas_*.csv"))[-1]
    df         = pd.read_csv(caminhoCSV)
    
    return df

In [3]:
def separarXeY(
    dfTreinamento:        pd.DataFrame,
    dfTeste:              pd.DataFrame, 
    featuresSelecionadas: Optional[list[str]] = None
) -> (pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series, pd.Series):
    
    # SELECIONANDO TODAS AS FEATURES SE FEATURESSELECIONADAS VIER NONE
    if featuresSelecionadas is None:
        featuresSelecionadas = df.columns.drop(["potenciaEspiao", "ataquePresente"])
    
    # SEPARANDO OS ARRAYS
    XTrain         = dfTreinamento[featuresSelecionadas]
    XTest          = dfTeste[featuresSelecionadas]
    yTrain         = dfTreinamento["ataquePresente"]
    yTest          = dfTeste["ataquePresente"]
    snrTrain       = dfTreinamento["snr"]
    snrTest        = dfTeste["snr"]
    potEspiaoTrain = dfTreinamento["potenciaEspiao"]
    potEspiaoTest  = dfTeste["potenciaEspiao"]
    
    return XTrain, XTest, yTrain, yTest, snrTrain, snrTest, potEspiaoTrain, potEspiaoTest

In [4]:
def deteccaoMachineLearning(
    XTrain:           pd.Series, 
    XTest:            pd.Series, 
    yTrain:           pd.Series, 
    objClassificador: Any
) -> (np.ndarray, np.float64, np.float64):

    # TREINANDO O CLASSIFICADOR
    objClassificador.fit(XTrain, yTrain)

    # PREDIZENDO OS DE TESTE
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[i]       = objClassificador.predict([amostraTeste.to_numpy()])[0]
        toc            = time()
        arrayTempos[0] = toc-tic
        
    # METRICAS DE TEMPO
    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [5]:
def deteccaoHassan(
    XTest: pd.Series
) -> (np.ndarray, np.float64, np.float64):
    
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[0]       = 1 if amostraTeste["E"] > amostraTeste["eta"] else 0
        toc            = time()
        arrayTempos[0] = toc-tic

    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [6]:
def deteccaoKapetanovic(
    XTest:     pd.Series, 
    threshold: float
) -> (np.ndarray, np.float64, np.float64):
    
    arrayTempos = np.zeros(len(XTest))
    yPred       = np.zeros(len(XTest))
    
    for i, (id, amostraTeste) in enumerate(XTest.iterrows()):
        tic            = time()
        yPred[0]       = 1 if amostraTeste["a1"]/amostraTeste["a2"] > threshold else 0
        toc            = time()
        arrayTempos[0] = toc-tic
        
    mediaTempo, desvioTempo = np.mean(arrayTempos), np.std(arrayTempos)
        
    return yPred, mediaTempo, desvioTempo

In [7]:
def predizerEsquemaDeteccao(
    esquemaDeteccao:      str,
    XTrain:               pd.Series, 
    XTest:                pd.Series, 
    yTrain:               pd.Series, 
    classificador:        Any,
    thresholdKapetanovic: float
) -> (np.ndarray, np.float64, np.float64):
    
    if esquemaDeteccao == "ML":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoMachineLearning(XTrain, XTest, yTrain, classificador)
    elif esquemaDeteccao == "Hassan":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoHassan(XTest)
    else:
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoKapetanovic(XTest, thresholdKapetanovic)
        
    return yPred, tempoMedioPredicao, tempoDesvioPredicao

In [8]:
def obterValoresUnicos(
    arraySNRs:      np.ndarray, 
    arrayPotsEspiao: np.ndarray
) -> (np.ndarray, np.ndarray):
    
    rangeSNRs      = np.unique(arraySNRs)
    rangePotEspiao = np.unique(arrayPotsEspiao)
    
    return rangeSNRs, rangePotEspiao

In [9]:
def calcularMetricas(
    yTest: pd.Series, 
    yPred: np.ndarray
) -> (np.float64, np.float64, np.float64, np.ndarray):
    
    acuracia       = accuracy_score(yTest, yPred)
    precisao       = precision_score(yTest, yPred)
    revocacao      = recall_score(yTest, yPred)
    matrizConfusao = confusion_matrix(yTest, yPred)
    
    return acuracia, precisao, revocacao, matrizConfusao

In [10]:
def obterDictNovoResultado(
    classificador:               Any, 
    matrizProbabilidadeDeteccao: np.ndarray, 
    matrizConfusao:              np.ndarray, 
    rangePotEspiao:              np.ndarray, 
    rangeSNRs:                   np.ndarray, 
    qtdUsuarios:                 int, 
    qtdAntenas:                  int, 
    qtdSimbolos:                 int, 
    featuresSelecionadas:        list[str], 
    acuracia:                    np.float64, 
    precisao:                    np.float64, 
    revocacao:                   np.float64, 
    tempoMedioPredicao:          np.float64, 
    tempoDesvioPredicao:         np.float64, 
    repetibilidadeDataset:       int
) -> dict[str, Any]:
    
    classificador = classificador.__class__.__name__ if type(classificador) is not str else classificador
    descricao = "Deteccao baseada em " + classificador + " com as features " + ", ".join(featuresSelecionadas)
    
    dictNovoResultado = {
        classificador + "_" + str(time()): {
            "qtdUsuarios":                 int(qtdUsuarios),
            "qtdAntenas":                  int(qtdAntenas),
            "qtdSimbolos":                 int(qtdSimbolos),
            "descricao":                   descricao,
            "repetibilidadeDataset":       int(repetibilidadeDataset),
            "acuracia":                    float(acuracia),
            "precisao":                    float(precisao),
            "revocacao":                   float(revocacao),
            "tempoMedioPredicao":          tempoMedioPredicao,
            "tempoDesvioPredicao":         tempoDesvioPredicao, 
            "featuresSelecionadas":        featuresSelecionadas,
            "rangePotEspiao":              rangePotEspiao.tolist(),
            "rangeSNRs":                   rangeSNRs.tolist(),
            "matrizProbabilidadeDeteccao": matrizProbabilidadeDeteccao.tolist(),
            "matrizConfusao":              matrizConfusao.tolist()
        }
    }
    
    return dictNovoResultado

In [11]:
def salvarNovosResultados(
    arrayResultados: list[dict], 
    arquivoSalvar:   Path
) -> None:
    
    for dictNovoResultado in arrayResultados:
        
        keyNova    = list(dictNovoResultado.keys())[0]
        valuesNovo = dictNovoResultado[list(dictNovoResultado.keys())[0]]
        
        with open(arquivoSalvar, mode="r+") as file:
            dictResultados = json.load(file)
            dictResultados[keyNova] = valuesNovo
            file.seek(0)
            json.dump(dictResultados, file, indent=4)
            file.truncate()

## Parâmetros iniciais

In [12]:
# PARAMETROS
classificador                   = DecisionTreeClassifier()
qtdUsuarios                     = 64
qtdAntenas                      = 256
qtdSimbolos                     = 300
repetibilidadeDataset           = 8
nucleosProcessador              = -1
thresholdKapetanovic            = 100
verbose                         = 100
featuresSelecionadasML          = ["E"]
featuresSelecionadasHassan      = ["E", "eta"]
featuresSelecionadasKapetanovic = ["a1", "a2"]
dirDatasets                     = Path("../../datasets/test_1/")

# INICIANDO O JSON QUE VAI GUARDAR OS RESULTADOS
dictResultados = {}
arquivoSalvar  = Path("../../results/test_1/detection_probability_" + str(time()).replace(".", "") + ".json")
with open(arquivoSalvar, mode="w") as file:
    json.dump(dictResultados, file)

In [24]:
# PARAMETROS INICIAIS
esquemaDeteccao      = "ML"
featuresSelecionadas = featuresSelecionadasML
qtdFolders           = 5

# MATRIZ QUE VAi GUARDAR O RESULTADO DE CADA ITERACAO DO KFOLD
matrizProbabilidadeDeteccaoCadaKFold = []

# ABRINDO OS DATAFRAMES DE TREINAMENTO E TESTE E SEPARANDO X E Y
dfTreinamento = abrirDataframe(dirDatasets, qtdUsuarios, qtdAntenas, datasetTreinamento=True)
dfTeste       = abrirDataframe(dirDatasets, qtdUsuarios, qtdAntenas, datasetTreinamento=False)
XTrainGlobal, XTestGlobal, yTrainGlobal, yTestGlobal, snrTrainGlobal, snrTestGlobal, potEspTrainGlobal, potEspTestGlobal = separarXeY(dfTreinamento, dfTeste, featuresSelecionadas)

# VALORES UNICOS DE SNR E POTENCIA DE ESPIAO PRA EU SABER O INDEX DAS MATRIZES DE RESULTADOS
rangeSNRs, rangePotEspiao = obterValoresUnicos(snrTestGlobal, potEspTestGlobal)

# OBTENDO INDICES DE TREINAMENTO E TESTE COM KFOLD (NAO DA PRA USAR DO JEITO FACIL PQ SAO DOIS DATAFRAMES DIFERENTES)
objKFold = KFold(n_splits=qtdFolders, shuffle=True)
pastasTreinamento, _ = zip(*objKFold.split(XTrainGlobal))
_, pastasTeste       = zip(*objKFold.split(XTestGlobal))

# RODANDO O KFOLD
for indexesTreinamento, indexesTeste in zip(pastasTreinamento, pastasTeste):

    # SEPARANDO X, Y, SNRs E POTENCIAS DO ESPIAO DE TREINAMENTO E TESTE
    XTrain, XTest           = XTrainGlobal.loc[indexesTreinamento], XTestGlobal.loc[indexesTeste]
    yTrain, yTest           = yTrainGlobal.loc[indexesTreinamento], yTestGlobal.loc[indexesTeste]
    snrTrain, snrTest       = snrTrainGlobal.loc[indexesTreinamento], snrTestGlobal.loc[indexesTeste]
    potEspTrain, potEspTest = potEspTrainGlobal.loc[indexesTreinamento], potEspTestGlobal.loc[indexesTeste]

    # MATRIZ QUE VAI GUARDAR OS RESULTADOS DESSA ITERACAO DO KFOLD
    matrizContadoraDeteccoes   = np.zeros(shape=(len(rangePotEspiao), len(rangeSNRs)))
    matrizContadoraOcorrencias = np.zeros(matrizContadoraDeteccoes.shape)

    # PREDICAO
    yPred, tempoMedioPredicao, tempoDesvioPredicao = predizerEsquemaDeteccao(esquemaDeteccao, XTrain, XTest, yTrain, classificador, thresholdKapetanovic)

    # AGREGO NAS MATRIZES CONTADORAS
    for (indexTesteAtual, predicaoAtual) in zip(yTest.index, yPred):

        # PEGANDO EM QUAL LINHA E EM QUAL COLUNA DA MATRIZ EU VOU COLOCAR O RESULTADO
        indexPotEsp = np.where(rangePotEspiao==potEspTest[indexTesteAtual])[0][0]
        indexSNR    = np.where(rangeSNRs==snrTest[indexTesteAtual])[0][0]

        # COLOCO NAS MATRIZES DE RESULTADOS
        matrizContadoraDeteccoes[indexPotEsp][indexSNR] += predicaoAtual
        matrizContadoraOcorrencias[indexPotEsp][indexSNR]  += 1

    # MATRIZ DA PROBABILIDADE DE DETECCAO EM CADA POT ESPIAO (LINHAS) E CADA SNR (COLUNAS)
    matrizProbabilidadeDeteccaoCadaKFold.append(matrizContadoraDeteccoes/matrizContadoraOcorrencias)

# MATRIZ DE PROBABILIDADE DE DETECCAO MEDIA E DESVIO EM CADA POT ESP E CADA SNR
matrizProbabilidadeDeteccaoMedia  = np.mean(matrizProbabilidadeDeteccaoCadaKFold, axis=0)
matrizProbabilidadeDeteccaoDesvio = np.std(matrizProbabilidadeDeteccaoCadaKFold, axis=0)


    
#     return matrizProbabilidadeDeteccaoMedia, matrizProbabilidadeDeteccaoDesvio

In [25]:
matrizProbabilidadeDeteccaoMedia

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00238095, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        0.992     , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.9875    , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.98823529,
        1.        , 1.        , 1.        , 0.98181818, 1.        ,
        1.        , 0.99166

In [ ]:
def rodarTesteDeteccao(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic=100):
    
    # PEGO O QUE E O QUE
    XTrain, XTest, yTrain, yTest, snrTrain, snrTest, potEspiaoTrain, potEspiaoTest = separarXeY(abrirDataframe(qtdUsuarios, "treinamento"), abrirDataframe(qtdUsuarios, "teste"), featuresSelecionadas)
    
    # VALORES UNICOS DE SNR E POTENCIA DE ESPIAO PRA EU SABER O INDEX DAS MATRIZES DE RESULTADOS
    rangeSNRs, rangePotEspiao = obterValoresUnicos(snrTest, potEspiaoTest)
    
    # MATRIZES QUE VAO GUARDAR OS RESULTADOS
    matrizProbabilidadeDeteccao = np.zeros(shape=(len(rangePotEspiao), len(rangeSNRs)))
    
    # PREDICAO
    if esquemaDeteccao == "ML":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoMachineLearning(XTrain, XTest, yTrain, classificador)
    elif esquemaDeteccao == "Hassan":
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoHassan(XTest)
    else:
        yPred, tempoMedioPredicao, tempoDesvioPredicao = deteccaoKapetanovic(XTest, thresholdKapetanovic)

    # AGREGO NA MATRIZ DE PROB DE DETECCAO
    for (indexTesteAtual, predicaoAtual) in zip(yTest.index, yPred):

        # PEGANDO EM QUAL LINHA E EM QUAL COLUNA DA MATRIZ EU VOU COLOCAR O RESULTADO
        indexPotEsp = np.where(rangePotEspiao==potEspiaoTest[indexTesteAtual])[0][0]
        indexSNR    = np.where(rangeSNRs==snrTest[indexTesteAtual])[0][0]

        # JA SOMO COM A PROBABILIDADE (DIVIDINDO PELA REPETIBILIDADE)
        matrizProbabilidadeDeteccao[indexPotEsp][indexSNR] += predicaoAtual/repetibilidadeDataset       
    
    # CONSIDERANDO QUE O DATASET ESTA BALANCEADO, HA MUITO MAIS REPETIBILIDADE QUANDO POTESP = 0
    matrizProbabilidadeDeteccao[0] /= (len(rangePotEspiao) - 1)
    
    # METRICAS
    acuracia, precisao, revocacao, matrizConfusao = calcularMetricas(yTest, yPred)
    
    # DICIONARIOS COM OS RESULTADOS 
    dictNovoResultado = obterDictNovoResultado(classificador, matrizProbabilidadeDeteccao, matrizConfusao, rangePotEspiao, rangeSNRs, qtdUsuarios, qtdAntenas, qtdSimbolos, featuresSelecionadas, acuracia, precisao, revocacao, tempoMedioPredicao, tempoDesvioPredicao, repetibilidadeDataset)
    
    return dictNovoResultado

## Rodando os testes de detecção com machine learning, hassan e kapetanovic

In [ ]:
esquemaDeteccao = "ML"
arrayResultados = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for classificador in classificadores for qtdUsuarios in rangeQtdUsuarios for featuresSelecionadas in combinacoesFeaturesML)
salvarNovosResultados(arrayResultados, arquivoSalvar)

In [ ]:
esquemaDeteccao      = "Hassan"
classificador        = esquemaDeteccao
featuresSelecionadas = ["E", "eta"]
arrayResultados      = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for qtdUsuarios in rangeQtdUsuarios)
salvarNovosResultados(arrayResultados, arquivoSalvar)

In [ ]:
esquemaDeteccao      = "Kapetanovic"
classificador        = esquemaDeteccao
featuresSelecionadas = ["a1", "a2"]
arrayResultados      = Parallel(verbose=verbose, n_jobs=nucleosProcessador)(delayed(rodarTesteDeteccao)(qtdUsuarios, featuresSelecionadas, classificador, esquemaDeteccao, repetibilidadeDataset, thresholdKapetanovic) for qtdUsuarios in rangeQtdUsuarios)
salvarNovosResultados(arrayResultados, arquivoSalvar)